In [1]:
import os

# set current dir to Mask3D dir
mask3d_dir = "/workspace/Mask3D_adapted"
os.chdir(mask3d_dir)

import logging
import os
from hashlib import md5
from uuid import uuid4
import hydra
from dotenv import load_dotenv
from omegaconf import DictConfig, OmegaConf
from trainer.trainer import InstanceSegmentation, RegularCheckpointing
from pytorch_lightning.callbacks import ModelCheckpoint
from utils.utils import (
    flatten_dict,
    load_baseline_model,
    load_checkpoint_with_missing_or_exsessive_keys,
    load_backbone_checkpoint_with_missing_or_exsessive_keys,
)
from datasets.utils import *
from pytorch_lightning import Trainer, seed_everything
import numpy as np

import logging
from itertools import product
from pathlib import Path
from random import random, sample, uniform
from typing import List, Optional, Tuple, Union
from random import choice
from copy import deepcopy
from random import randrange


import numpy
import torch
from datasets.random_cuboid import RandomCuboid

import albumentations as A
import numpy as np
import scipy
import volumentations as V
import yaml

# from yaml import CLoader as Loader
from torch.utils.data import Dataset
from datasets.scannet200.scannet200_constants import (
    SCANNET_COLOR_MAP_200,
    SCANNET_COLOR_MAP_20,
)
MULTISCAN_COLOR_MAP = {0: (0.0, 0.0, 0.0),
 1: (191, 246, 112),
 2: (110, 239, 148),
 255: (0.0, 0.0, 0.0),}
from datasets.scannetpp.scannetpp_constants import (SCANNETPP_COLOR_MAP)

# models 
from models.mask3d import *

def elastic_distortion(pointcloud, granularity, magnitude):
    """Apply elastic distortion on sparse coordinate space.

    pointcloud: numpy array of (number of points, at least 3 spatial dims)
    granularity: size of the noise grid (in same scale[m/cm] as the voxel grid)
    magnitude: noise multiplier
    """
    blurx = np.ones((3, 1, 1, 1)).astype("float32") / 3
    blury = np.ones((1, 3, 1, 1)).astype("float32") / 3
    blurz = np.ones((1, 1, 3, 1)).astype("float32") / 3
    coords = pointcloud[:, :3]
    coords_min = coords.min(0)

    # Create Gaussian noise tensor of the size given by granularity.
    noise_dim = ((coords - coords_min).max(0) // granularity).astype(int) + 3
    noise = np.random.randn(*noise_dim, 3).astype(np.float32)

    # Smoothing.
    for _ in range(2):
        noise = scipy.ndimage.filters.convolve(
            noise, blurx, mode="constant", cval=0
        )
        noise = scipy.ndimage.filters.convolve(
            noise, blury, mode="constant", cval=0
        )
        noise = scipy.ndimage.filters.convolve(
            noise, blurz, mode="constant", cval=0
        )

    # Trilinear interpolate noise filters for each spatial dimensions.
    ax = [
        np.linspace(d_min, d_max, d)
        for d_min, d_max, d in zip(
            coords_min - granularity,
            coords_min + granularity * (noise_dim - 2),
            noise_dim,
        )
    ]
    interp = scipy.interpolate.RegularGridInterpolator(
        ax, noise, bounds_error=0, fill_value=0
    )
    pointcloud[:, :3] = coords + interp(coords) * magnitude
    return pointcloud

/opt/conda/envs/mask3d_cuda113/lib/python3.10/site-packages/MinkowskiEngine-0.5.4-py3.10-linux-x86_64.egg/MinkowskiEngine/__init__.py:36: UserWarning: The environment variable `OMP_NUM_THREADS` not set. MinkowskiEngine will automatically set `OMP_NUM_THREADS=16`. If you want to set `OMP_NUM_THREADS` manually, please export it on the command line before running a python script. e.g. `export OMP_NUM_THREADS=12; python your_program.py`. It is recommended to set it below 24.
  warnings.warn(


In [2]:
import MinkowskiEngine as ME
import numpy as np
import torch
from random import random
from datasets.semseg import SemanticSegmentationArticulationDataset

In [3]:
dataaset = SemanticSegmentationArticulationDataset(
        dataset_name="scenefun3d",
        data_dir = "data/processed/scenefun3d",
        label_db_filepath = "data/processed/scenefun3d/label_database.yaml",
        color_mean_std = "data/processed/scenefun3d/color_mean_std.yaml",
        mode = "validation",
        add_colors = True,
        add_normals = False,
        add_raw_coordinates = True,
        add_instance = True,
        num_labels = 3,
        data_percent = 1.0,
        ignore_label = 255,
        volume_augmentations_path = None,
        image_augmentations_path = None,
        instance_oversampling=0,
        place_around_existing=False,
        max_cut_region=0,
        point_per_cut=0,
        flip_in_center=False,
        noise_rate=0.0,
        resample_points=0.0,
        cache_data=False,
        add_unlabeled_pc=False,
        task="instance_segmentation",
        cropping=False,
        cropping_args=None,
        is_tta=False,
        crop_min_size=20000,
        crop_length=6.0,
        cropping_v1=True,
        reps_per_epoch=1,
        area=-1,
        on_crops=False,
        eval_inner_core=-1,
        filter_out_classes=[0, 255],
        label_offset=0,
        add_clip=False,
        is_elastic_distortion=True,
        color_drop=0.0,
        load_articulation = True
)
collater = VoxelizeCollate(
        ignore_label=255,
        voxel_size=0.01,
        mode="train_mode",
        small_crops=False,
        very_small_crops=False,
        batch_instance=False,
        probing=False,
        task="instance_segmentation",
        ignore_class_threshold=100,
        filter_out_classes=[0, 255],
        label_offset=0,
        num_queries=150,
        load_articulation=True
)

self.use_coarse_to_fine:  False
type(self.use_coarse_to_fine):  <class 'bool'>


/workspace/Mask3D_adapted/datasets/semseg.py:920: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  file = yaml.load(f)


In [4]:
for i in range(len(dataaset)):
    if dataaset[i][3] == '421393':
        print(i)
        break
    

1


In [5]:
data_batch = collater([dataaset[1]])

In [10]:
data_batch[1][0]

{'labels': tensor([1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2]),
 'masks': tensor([[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ...,  True,  True,  True]]),
 'segment_mask': tensor([[True],
         [True],
         [True],
         [True],
         [True],
         [True],
         [True],
         [True],
         [True],
         [True],
         [True]]),
 'articulations': {'origin': tensor([[-9.4077e-01, -7.0117e-01,  1.4104e+02],
          [ 9.3317e-01, -4.7149e-01,  1.4151e+02],
          [ 9.3034e-01, -4.7626e-01,  1.4134e+02],
          [ 9.2718e-01, -4.7698e-01,  1.4116e+02],
          [ 9.3130e-01, -4.6246e-01,  1.4098e+02],
          [ 5.0210e-01, -8.6607e-01,  1.4188e+02],
          [ 1.1668e+00, -1.12

In [9]:
test_file = "/workspace/Mask3D_adapted/data/processed/multiscan/instance_gt/validation/scene_00002_01.txt"
test = np.loadtxt(test_file).astype(np.int32)
np.unique(test )

array([   1, 1005, 1007, 1027, 1030, 1031], dtype=int32)

In [15]:
np.unique(data_batch[0].original_labels[0][:, 1])

array([ 0,  4,  6, 26, 29, 30], dtype=int32)

In [8]:
np.unique(dataaset[1][2][:, 1])

array([ 0,  4,  6, 26, 29, 30], dtype=int32)

In [15]:
masks = data_batch[1][0]['masks']

inst_gt = np.zeros(masks.shape[1])
for i in range(masks.shape[0]):
    inst_mask = masks[i].numpy()
    inst_gt[inst_mask] = i

In [16]:
test = data_batch[0].coordinates[:, 1:].numpy()

In [17]:
test.shape

(185682, 3)

In [18]:
inst_gt.shape

(185682,)

In [19]:
np.save("test2.npy", test)

In [20]:
np.save("inst.npy", inst_gt)

In [11]:
data_batch

(<datasets.utils.NoGpu at 0x7fc5f0b37d00>,
 [{'labels': tensor([1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
   'masks': tensor([[False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           ...,
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False]]),
   'segment_mask': tensor([[True],
           [True],
           [True],
           [True],
           [True],
           [True],
           [True],
           [True],
           [True],
           [True],
           [True],
           [True],
           [True],
           [True],
           [True],
           [True],
           [True],
           [True],
           [True],
           [True],
           [True],
           [True],
           [True],
           [True